In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from models import Generator, Discriminator
from PIL import Image
from torchvision import datasets, transforms
from torch.autograd import Variable
import torchvision.utils as vutils
import re
import os

In [2]:
test_name = "try13"
data = "../data/training_data"
batch_size = 300
lr = 0.0005
betas = (0.5, 0.99)
weight_decay = 0.00001
epoch_test_interval = 1
batch_test_interval = 10000

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == "cuda":
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
print(f"Will run on {device}.")

torch.manual_seed(0)
noise = Variable(torch.randn(1, 100, 1, 1)).to(device)
test_noise = Variable(torch.randn(9, 100, 1, 1)).to(device)

document_path = f"../data/documentation/{test_name}"
progress_path = f"{document_path}/gan_progress"
os.makedirs(progress_path, exist_ok=True)

def pil_loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGBA')


def get_flag_loader(dir=data, batch_size=batch_size, shuffle=True):
    transform = transforms.ToTensor()
    flag_dataset = datasets.ImageFolder(
        root=dir, transform=transform, loader=pil_loader)
    flag_loader = torch.utils.data.DataLoader(
        dataset=flag_dataset, batch_size=batch_size, shuffle=shuffle)

    return flag_loader


def train(data_loader, epoch):
    D.train()
    G.train()

    for batch_idx, (data, _) in enumerate(data_loader):
        n_samples = data.size(dim=0)

        #######################
        # Train Discriminator #
        #######################

        # Zero out gradients on discriminator
        D.zero_grad()

        # Load real flag data, run through discriminator and compute BCE loss
        # against target vector of all ones, because the flags are legit
        real_data = Variable(data).to(device)
        output = D(real_data)
        real_target = Variable(torch.ones(n_samples)).to(device)
        real_error = loss(output.squeeze(), real_target)

        # Get normally distributed noise and feed to generator to create fake
        # flag data. Run fake flag data through discriminator and compute BCE
        # loss against target vector of all zeros, because data is fake. Detach
        # to avoid training generator on these labels
        noise = Variable(torch.randn(n_samples, 100, 1, 1)).to(device)
        fake_data = G(noise)
        output = D(fake_data.detach()).to(device)
        fake_target = Variable(torch.zeros(n_samples)).to(device)
        fake_error = loss(output.squeeze(), fake_target)

        # Compute accumulated gradient based on real and fake data to update
        # discriminator weights
        d_error = real_error + fake_error
        d_error.backward()
        d_optim.step()

        ###################
        # Train Generator #
        ###################

        # Zero out gradients on generator
        G.zero_grad()

        # Run fake flag data through discriminator and compute BCE loss against
        # target vector of all ones. We want to fool the discriminator, so
        # pretend the mapped data is genuine
        output = D(fake_data)
        g_error = loss(output.squeeze(), real_target)

        # Compute new gradients from discriminator and update weights of the generator
        g_error.backward()
        g_optim.step()

        if epoch % epoch_test_interval == 0 and batch_idx % batch_test_interval == 0:
            # Logging
            log_text = '({:02d}, {:02d}) \tLoss_D: {:.6f} \tLoss_G: {:.6f}'.format(epoch, batch_idx, d_error.data, g_error.data)
            print(log_text)
            file = open(f"{document_path}/loss.txt", "a+")
            file.write(log_text + "\n")
            
            # Test Generator
            with torch.no_grad():
                sample = G(test_noise).detach().cpu()
            grid = vutils.make_grid(sample, padding=2, normalize=True, nrow=3)
            img = np.transpose(grid, (1,2,0)).numpy()
            plt.clf()
            imgplot = plt.imshow(img)
            plt.title(f"Epoch {epoch}")
            plt.savefig(f"{progress_path}/{epoch}.png")

Will run on cuda.


In [4]:
G, D = Generator().to(device), Discriminator().to(device)
loss = nn.BCELoss()
g_optim = optim.AdamW(G.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
d_optim = optim.AdamW(D.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)

flag_loader = get_flag_loader()
noise = Variable(torch.randn(1, 100, 1, 1))

epoch = 0

**Optional:** Load previous network progress.

In [5]:
D_file_name = "../data/documentation/try16/net_D_e748.pth"
G_file_name = "../data/documentation/try16/net_G_e748.pth"

D.load_state_dict(torch.load(D_file_name))
G.load_state_dict(torch.load(G_file_name))

epoch = int(re.findall(r"net\_G\_e(\d+)", G_file_name)[0])

In [ ]:
while True:
    train(data_loader=flag_loader, epoch=epoch)
    epoch += 1

(548, 00) 	Loss_D: 0.008919 	Loss_G: 14.757826
(549, 00) 	Loss_D: 0.000177 	Loss_G: 10.585363
(550, 00) 	Loss_D: 0.002067 	Loss_G: 12.778118
(551, 00) 	Loss_D: 0.000024 	Loss_G: 11.688669
(552, 00) 	Loss_D: 0.000042 	Loss_G: 11.379169
(553, 00) 	Loss_D: 0.000637 	Loss_G: 24.427641
(554, 00) 	Loss_D: 0.000714 	Loss_G: 9.207132
(555, 00) 	Loss_D: 0.000682 	Loss_G: 9.294248
(556, 00) 	Loss_D: 0.002192 	Loss_G: 10.770787
(557, 00) 	Loss_D: 0.000604 	Loss_G: 20.357693
(558, 00) 	Loss_D: 0.000107 	Loss_G: 13.682560
(559, 00) 	Loss_D: 0.000406 	Loss_G: 8.906823
(560, 00) 	Loss_D: 0.001476 	Loss_G: 8.075767
(561, 00) 	Loss_D: 0.000347 	Loss_G: 8.846148
(562, 00) 	Loss_D: 0.001371 	Loss_G: 8.479985
(563, 00) 	Loss_D: 0.000656 	Loss_G: 8.389080
(564, 00) 	Loss_D: 0.000301 	Loss_G: 9.084194
(565, 00) 	Loss_D: 0.000030 	Loss_G: 21.877491
(566, 00) 	Loss_D: 0.000072 	Loss_G: 14.355715
(567, 00) 	Loss_D: 0.000074 	Loss_G: 11.325435
(568, 00) 	Loss_D: 0.000064 	Loss_G: 13.153550
(569, 00) 	Loss_D: 0.

**Optional:** Save networks.

In [7]:
torch.save(G.state_dict(), f"{document_path}/net_G_e{epoch}.pth")
torch.save(D.state_dict(), f"{document_path}/net_D_e{epoch}.pth")